In [1]:
import json
from twarc_csv import DataFrameConverter
from twarc.expansions import ensure_flattened
import pandas as pd
from twarc_csv import dataframe_converter
from twarc_csv import CSVConverter, DataFrameConverter
import math
import emoji
from operator import itemgetter
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
from advertools import extract_emoji
import numpy as np
import pickle

**Manual Input**: Set the path.

In [2]:
path='C:/Users/acoub/OneDrive/Desktop/DSDM/Thesis/ChileGov/methodology_feminism'
sep='/'

In [3]:
path_scripts = path+sep+'initial'
path_data= path+sep+'data'

In [4]:
converter = DataFrameConverter()
data = []
with open(path+sep+"data"+sep+"first_search.json") as f:
    for line in f:
        data.append(json.loads(line))
df = converter.process(data)

In [5]:
df.shape

(186, 74)

In [6]:
def hash_retrieve(df):
    """
    df : dataframe of tweets
    Description: 
        The function takes as an object a df of tweets obtained via twarc and 
        returns a generator object.
    
    """

    for line, id in zip(df['entities.hashtags'], df['id']):
        if pd.isna(line):
            continue
        line = line.strip()
        data = json.loads(line)
        for hashtag in ensure_flattened(data):
            yield [hashtag['tag'], id]

In [7]:
hashtags_df = pd.DataFrame(list(hash_retrieve(df)),
                 columns=['tweet_hashtags', 'id'])

hashtags_df = hashtags_df.groupby('id')['tweet_hashtags'].apply(lambda x: ','.join(x))
df = df.merge(hashtags_df, how='left', left_on='id', right_on='id')
df['tweet_hashtags']=df['tweet_hashtags'].str.lower()

In [8]:
hasht=[]
for i in df.index:
    try:
        hasht.append(df['tweet_hashtags'][i].split(","))
    except:
        hasht.append([])
    
    
df['list_hashtags']=hasht

In [9]:
hashtags=[]
for i in df.index:
    hashtags+=df['list_hashtags'][i]

**Look output**: 100 most used hashtag for your topic in tweets geolocated in Chile.

In [10]:
from collections import Counter
count = Counter(hashtags)
  
# most_common() produces k frequently encountered
# input values and their respective counts.
most_occur = count.most_common(100)
  
print(most_occur)

[('8m', 26), ('feminismo', 17), ('8marzo2022', 10), ('8m2022', 7), ('boricpresidentedechile', 6), ('chile', 6), ('feminista', 6), ('cambiodemando', 5), ('diadelamujer', 5), ('8marzo', 5), ('boric', 4), ('feministas', 4), ('mujeres', 3), ('8demarzo', 3), ('internationalwomensday', 3), ('cambiodemando2022', 2), ('boricpresidente', 2), ('machismo', 2), ('sanmiguel', 2), ('seguimos', 2), ('feminazis', 2), ('nomerepresentan', 2), ('womensday', 2), ('cuba', 2), ('chileestadeluto', 1), ('acosadorboric', 1), ('parlamentofeminista', 1), ('chaopinera', 1), ('cerrocastillo', 1), ('gabinetefeminista', 1), ('11demarzo', 1), ('discipulogranfinal', 1), ('lastarria', 1), ('lamanada', 1), ('constituyentechile', 1), ('abortolegalseguroygratuito', 1), ('seraley', 1), ('derechoalaautonomia', 1), ('aborto', 1), ('abortolegal', 1), ('ahoraysiemprehermanas', 1), ('antofagasta', 1), ('coctelería', 1), ('barwoman', 1), ('bartender', 1), ('cocktails', 1), ('barmaid', 1), ('feminist', 1), ('girlspower', 1), ('am

In [11]:
import numpy as np
import pandas as pd
import emoji
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer   
from nltk.stem.wordnet import WordNetLemmatizer
import nltk
import re
import matplotlib.pyplot as plt
import seaborn as sns
import string

stopwords_eng =stopwords.words("english")
stopwords_es =stopwords.words("spanish")
stopwords_all=stopwords_eng+stopwords_es

def cleanTweets(s,remove_hash):
    #Function to clean tweets, for now i am keeping emojis and hashtags. Alternative version
    if type(s)==np.float:
        return ""
    #Demojize text
#    s=emoji.demojize(s,delimiters=("", " "))
    
    #Remove new lines, etc.
    s = s.replace(r'<lb>', "\n")
    s = s.replace(r'<tab>', "\i")
    s = re.sub(r'<br */*>', "\n", s)
    s = s.replace("&lt;", "<").replace("&gt;", ">").replace("&amp;", "&")
    s = s.replace("&amp;", "&")
    s = s.replace("\n", " ")
    s = s.replace("\\n", " ")
    
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    s= emoji_pattern.sub(r'', s)
    
    
    # markdown urls
    s = re.sub(r'\(https*://[^\)]*\)', "", s)
    # normal urls
    s = re.sub(r'https*://[^\s]*', "", s)
    #s = re.sub(r'_+', ' ', s)
    s = re.sub(r'"+', '"', s)
    #Remove punctuation    
    s = re.sub('[()!?]', ' ', s)
    s = re.sub('\[.*?\]',' ', s)
    s = re.sub('\[,*?\]',' ', s)
    # custom removals
    #s = re.sub(r'@[A-Za-z0-9_]+', "", s) # replace mentions
    s = re.sub(r':[^:]+','[emoji]',s) # remove demojized text
    s= re.sub(r'[0-9]','',s)# remove digits
    
    s = s.translate(str.maketrans('', '', '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~'))
    
    s=s.lower()
    if remove_hash==True:
        s = re.sub(r'#[A-Za-z0-9_]+', "", s)
    
    #Remove stopwords
    s = ' '.join([word for word in s.split() if word not in stopwords_all])
    
    return str(s)

In [12]:
import unidecode
df['cleaned_text'] = [cleanTweets(text,False) for text in df['text']]
df['cleaned_text']= df['cleaned_text'].apply(unidecode.unidecode)

<ipython-input-11-8fa97597cc03>:19: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if type(s)==np.float:


**Look output**: 100 most used word for your topic in tweets geolocated in Chile.

In [13]:
df['cleaned_text']=df['cleaned_text'].fillna("")
string=" ".join(list(df['cleaned_text']))
from collections import Counter
split_it = string.split()
Counter = Counter(split_it)
most_occur = Counter.most_common(200)
  
print(most_occur)

[('feminista', 77), ('feminismo', 51), ('mujeres', 44), ('feministas', 35), ('#m', 33), ('si', 20), ('mujer', 19), ('chile', 18), ('q', 17), ('ser', 16), ('#marzo', 16), ('gobierno', 15), ('#feminismo', 15), ('marcha', 15), ('dia', 14), ('solo', 13), ('asi', 11), ('hoy', 11), ('hombres', 11), ('igualdad', 11), ('todas', 9), ('@gabrielboric', 9), ('hombre', 9), ('lucha', 9), ('tan', 8), ('parte', 8), ('genero', 8), ('siempre', 7), ('mundo', 7), ('poder', 7), ('derechos', 7), ('hace', 6), ('mama', 6), ('#boricpresidentedechile', 6), ('movimiento', 6), ('ver', 6), ('nuevo', 6), ('#chile', 6), ('#cambiodemando', 6), ('#feminista', 6), ('anos', 6), ('gracias', 6), ('verguenza', 6), ('via', 5), ('cosas', 5), ('pais', 5), ('presidente', 5), ('mas', 5), ('justo', 5), ('aborto', 5), ('lindo', 5), ('violencia', 5), ('trabajo', 5), ('santiago', 5), ('#diadelamujer', 5), ('dan', 4), ('gabinete', 4), ('nueva', 4), ('toda', 4), ('gusta', 4), ('van', 4), ('foto', 4), ('x', 4), ('mucha', 4), ('mejor',

**Manual input**: Looking the most common words in chilean context, decide if you want to add new words to the list and put all the words in the list (the words that you used in the first search and also the words that you want to add).
  
Also add the start and end date for your final data set.

In [14]:
list_of_words=['feminismo','feminista','feministas','sexismo','sexista','machismo','sororidad','genero']

In [15]:
filename = path_data+sep+'keywords.sav'
pickle.dump((list_of_words), open(filename, 'wb'))

In [16]:
filename = path_data+sep+'start_search_date.sav'
start_date= pickle.load(open(filename, 'rb'))

filename = path_data+sep+'end_search_date.sav'
end_date= pickle.load(open(filename, 'rb'))

**Manual input:** Add words related with your country of interest separated by an OR.

In [17]:
str_country="(chile OR santiago OR iquique OR arica OR piñera OR pinera OR boric OR kast OR valparaiso OR antofagasta OR colchane OR copiapo OR coquimbo OR rancagua OR talca OR conce OR temuco OR puerto montt OR valdivia OR coyhaique OR punta arenas OR weon OR weona)"

In [18]:
filename = path_data+sep+'country_string.sav'
pickle.dump((str_country), open(filename, 'wb'))

Read the output and follow the instruction in case that you have to remove some words.

In [19]:
query=""
for word in list_of_words:
    query+= word + " OR "
query=query[:len(query)-4]

query_place="("+query+")"+ " -is:retweet place_country:CL"
query_wrelated="("+query+")"+ str_country + " -is:retweet"

if len(query_place)<=1024 and len(query_wrelated)<=1024:
    print("Ready to run the next cell and search in twarc.")
else:
    print("Query too long, reduce the list of words.")

Ready to run the next cell and search in twarc.


In [20]:
with open(path_data+sep+"second_search.txt", 'w') as f:
    f.write(query_place)
    f.write("\n")
    f.write(query_wrelated)

**Manual revision:** Run the following two cells to know the total number of tweets to download and see if does not exceed the limit of your twarc acount. If is OK run also the next cell.

In [21]:
file=path_data+sep+"second_search.txt"
%cd {path_data}
! twarc2 searches --archive --counts-only --granularity day --start-time {start_date} --end-time {end_date} {file} second_search_count.csv

C:\Users\acoub\OneDrive\Desktop\DSDM\Thesis\Methodology\data



  0%|          | Processed 0/2 lines of input file [00:00<?, ?it/s]
100%|##########| Processed 2/2 lines of input file [00:01<00:00,  1.38it/s]
100%|##########| Processed 2/2 lines of input file [00:02<00:00,  1.39s/it]


In [22]:
count=pd.read_csv(path_data+sep+'second_search_count.csv',encoding = "ISO-8859-1")
print("Tweets to download: "+str(count['day_count'].sum()))

Tweets to download: 2444


**Download the tweets:** (Only if you have enought limit in your twarc account)

In [23]:
%cd {path_data}
! twarc2 searches --archive --start-time {start_date} --end-time {end_date} {file} second_search.json

C:\Users\acoub\OneDrive\Desktop\DSDM\Thesis\Methodology\data



  0%|          | Processed 0/2 lines of input file [00:00<?, ?it/s]
100%|##########| Processed 2/2 lines of input file [00:06<00:00,  3.49s/it]
100%|##########| Processed 2/2 lines of input file [00:57<00:00, 28.96s/it]
